## **Atividade - Redes Neurais**

Construção de Rede Neural Convolucional para reconhecimento de dígitos dispostos em linguagem natural

Escola Politécnica da Universidade de Pernambuco - UPE/POLI

**Disciplina: Redes Neurais / 2021.1**

**Equipe**
- Matheus Phelipe (mpap@ecomp.poli.br)
- Murilo Stoldoni (mcs2@ecomp.poli.br)
- Nilton Vieira (nvs@ecomp.poli.br)
- Richard Jeremias (rjmr@ecomp.poli.br)

Definições básicas

Objetivos

✅ 1) Número de camadas convolucionais e filtros ( pelo menos 4 variações)

✅ 2) Dropout - 4 valores

✅ 3) Testar diferentes algoritmos de otimização - SGD, RMSprop e Adam (pelo menos 3)

✅ 4) Variar o número de épocas ( mínimo de 3)

✅ 5) Variar a taxa de aprendizagem ( mínimo de 3)

✅ 6) Incrementar o tamanho do batch ( mínimo de 3)


### **Import inicial** 


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from matplotlib import pyplot as plt 
%matplotlib inline

from keras import backend
backend.set_image_data_format('channels_first')

seed = 7
np.random.seed(seed)

DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [ ]:
import logging
log = logging.getLogger()
log.setLevel(logging.DEBUG)

### **Fase 01: Import da base de dados** 


In [ ]:
from keras.datasets import mnist
[xTrain, yTrain], [xTest, yTest] = mnist.load_data()
logging.info(f'Base de dados carregada.')

INFO:root:Base de dados carregada.


### **Fase 02: Transformação dos dados** 


In [ ]:
from keras.utils import np_utils
#Conjunto de treino deve se adequar as dimensões da imagem
def reshapeInput(InputSet ,isGrayCh = 1, width = 28, heigh = 28):
  #0 -> treino, 1 -> teste
  InputSet[0] = InputSet[0].reshape(InputSet[0].shape[0], isGrayCh, width, heigh).astype('float32')
  InputSet[1] = InputSet[1].reshape(InputSet[1].shape[0], isGrayCh, width, heigh).astype('float32')
  logging.info(f'Redimensionando matriz de entrada (qtde de amostras, canal, largura, altura)\n Treino: {InputSet[0].shape}\n Teste: {InputSet[1].shape}')
  return (InputSet[0], InputSet[1])

def setMaxOutputRange(outputSet, range = 255):
  #0 -> treino, 1 -> teste
  outputSet[0] = outputSet[0] / range
  outputSet[1] = outputSet[1] / range
  logging.info(f'Valores de saída foram delimitados pelo limiar de {range}.')
  return (outputSet[0], outputSet[1])

def convertToMultiClassMatrix(outputSet):
  #0 -> treino, 1 -> teste
  outputSet[0] = np_utils.to_categorical(outputSet[0])
  outputSet[1] = np_utils.to_categorical(outputSet[1])
  logging.info(f'Vetor de rótulos convertido para matriz com valores categóricos.')
  return (outputSet[0], outputSet[1])

[xTrain, xTest] = reshapeInput([xTrain, xTest])
[xTrain, xTest] = setMaxOutputRange([xTrain, xTest])
[yTrain, yTest] = convertToMultiClassMatrix([yTrain, yTest])

INFO:root:Redimensionando matriz de entrada (qtde de amostras, canal, largura, altura)
 Treino: (60000, 1, 28, 28)
 Teste: (10000, 1, 28, 28)
INFO:root:Valores de saída foram delimitados pelo limiar de 255.
INFO:root:Vetor de rótulos convertido para matriz com valores categóricos.


**Definição da quantidade de rótulos**

In [ ]:
labelsAmount = yTest.shape[1]
logging.info(f'Quantidade de classes a serem processadas: {labelsAmount}.')

INFO:root:Quantidade de classes a serem processadas: 10.


### **Fase 03: Construção do modelo** 


In [ ]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Dropout, Flatten

Classe `CNNBuilder` permite criar arquitetura de rede com camadas personalizadas.

In [ ]:
class CNNBuilder():

  def __init__(self, modelName = 'Sequential'):
    self.model = Sequential(name = modelName)

  def getModelInstance(self):
    return self.model

  #Adição de camada convolucional
  def addConv2DLayer(self, featureMaps = 30, kernelSize = [5, 5], 
               dataShape = [1, 28, 28], activationFun = 'relu'):
    self.model.add(Conv2D(featureMaps, kernel_size= kernelSize, input_shape = dataShape, activation= activationFun)) #Número de feature maps 

  #Adição de camada Pooling
  def addMaxPooling2DLayer(self, poolingSize = [2,2]):
    self.model.add(MaxPooling2D(pool_size= poolingSize))

  #Adição de camada Pooling
  def addAvgPooling2DLayer(self, poolingSize = [2,2]):
    
    self.model.add(AveragePooling2D(pool_size= poolingSize[0]))

  #Adição de camada drop out, evitar overfitting
  def addDropoutLayer(self, dropOutRate = 0.2):
    self.model.add(Dropout(dropOutRate))
    
  #Flatten das features
  def addFlattenLayer(self):
    self.model.add(Flatten())

  #Adição de camadas densamente conectadas
  def addDenseLayer(self, neuronsAmount = 16, activation= 'softmax', name = 'Predict'):
    self.model.add(Dense(neuronsAmount, activation= 'softmax', name = 'Predict'))
  
  def addMultipleDenseLayers(self, neuronsAmount = 128, labelsAmount = 10, layers = 3, activationFun = 'relu'): 
    if(layers <= 0):
      logging.error('Quantidade de camadas inválida')
      return None

    for i in range(0, layers):
      self.model.add(Dense(neuronsAmount, activationFun))
      neuronsAmount = neuronsAmount / 2
    self.addDenseLayer(neuronsAmount = labelsAmount) #ultima camada

  def compileModel(self, lossFun = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy']):
    self.model.compile(loss= lossFun, optimizer= optimizer, metrics= metrics)


**Modelos CNN personalizados**

In [ ]:
def model1():
  cnnModel = CNNBuilder('Modelo_01')
  cnnModel.addConv2DLayer(featureMaps = 40) #se n passar nd, serão usados valores default
  cnnModel.addMaxPooling2DLayer()
  cnnModel.addDropoutLayer()
  cnnModel.addFlattenLayer()
  cnnModel.addMultipleDenseLayers()
  opt = keras.optimizers.Adam(learning_rate=0.015)
  cnnModel.compileModel(optimizer= opt)
  logging.info(f'Definida taxa de aprendizado de: {round(opt.learning_rate.numpy()*100,2)}%')
  return cnnModel.model

model1 = model1()
model1.summary()

INFO:root:Definida taxa de aprendizado de: 1.5%


Model: "Modelo_01"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 40, 24, 24)        1040      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 40, 12, 12)       0         
 2D)                                                             
                                                                 
 dropout_11 (Dropout)        (None, 40, 12, 12)        0         
                                                                 
 flatten_11 (Flatten)        (None, 5760)              0         
                                                                 
 dense_33 (Dense)            (None, 128)               737408    
                                                                 
 dense_34 (Dense)            (None, 64)                8256      
                                                         

In [ ]:
def model2(): #O que nilton fez fica assim
  cnnModel = CNNBuilder('Modelo_02')#vc pode nomear o modelo se quiser
  cnnModel.addConv2DLayer(featureMaps = 120, kernelSize= [5, 5]) #se n passar nd, serão usados valores default
  cnnModel.addAvgPooling2DLayer(poolingSize= [3,3])
  cnnModel.addConv2DLayer(featureMaps=40, kernelSize= [4,4])
  cnnModel.addMaxPooling2DLayer(poolingSize= [2,2])
  cnnModel.addDropoutLayer(dropOutRate=0.25)
  cnnModel.addFlattenLayer()
  cnnModel.addMultipleDenseLayers(neuronsAmount=128, layers = 3, activationFun= 'relu')
  opt = keras.optimizers.SGD(learning_rate=0.02)
  cnnModel.compileModel(optimizer=opt)
  logging.info(f'Definida taxa de aprendizado de: {round(opt.learning_rate.numpy()*100,2)}%')
  return cnnModel.model

model2 = model2()
model2.summary()

INFO:root:Definida taxa de aprendizado de: 2.0%


Model: "Modelo_02"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_42 (Conv2D)          (None, 120, 24, 24)       3120      
                                                                 
 average_pooling2d_16 (Avera  (None, 120, 8, 8)        0         
 gePooling2D)                                                    
                                                                 
 conv2d_43 (Conv2D)          (None, 40, 5, 5)          76840     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 40, 2, 2)         0         
 g2D)                                                            
                                                                 
 dropout_15 (Dropout)        (None, 40, 2, 2)          0         
                                                                 
 flatten_15 (Flatten)        (None, 160)               0 

In [ ]:
def model3():
  cnnModel = CNNBuilder('Modelo_03')
  cnnModel.addConv2DLayer(featureMaps = 25, kernelSize= [5, 5], activationFun= 'selu')
  cnnModel.addConv2DLayer(featureMaps = 25, kernelSize= [4, 4], activationFun= 'elu')
  cnnModel.addConv2DLayer(featureMaps = 25, kernelSize= [3, 3], activationFun= 'relu')
  cnnModel.addAvgPooling2DLayer(poolingSize= [2,2])
  cnnModel.addDropoutLayer(dropOutRate=0.15)
  cnnModel.addFlattenLayer()
  cnnModel.addMultipleDenseLayers(neuronsAmount=144, layers = 3, activationFun= 'sigmoid')
  opt = keras.optimizers.RMSprop(learning_rate=0.001)
  cnnModel.compileModel(lossFun= 'categorical_crossentropy', optimizer=opt)
  logging.info(f'Definida taxa de aprendizado de: {round(opt.learning_rate.numpy()*100,2)}%')
  return cnnModel.model

model3 = model3()
model3.summary()

INFO:root:Definida taxa de aprendizado de: 0.1%


Model: "Modelo_03"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_35 (Conv2D)          (None, 25, 24, 24)        650       
                                                                 
 conv2d_36 (Conv2D)          (None, 25, 21, 21)        10025     
                                                                 
 conv2d_37 (Conv2D)          (None, 25, 19, 19)        5650      
                                                                 
 average_pooling2d_15 (Avera  (None, 25, 9, 9)         0         
 gePooling2D)                                                    
                                                                 
 dropout_13 (Dropout)        (None, 25, 9, 9)          0         
                                                                 
 flatten_13 (Flatten)        (None, 2025)              0         
                                                         

In [ ]:
def model4():
  cnnModel = CNNBuilder('Modelo_04')
  cnnModel.addConv2DLayer(featureMaps = 30, kernelSize= [5, 5], activationFun= 'elu')
  cnnModel.addConv2DLayer(featureMaps = 30, kernelSize= [5, 5], activationFun= 'selu')
  cnnModel.addConv2DLayer(featureMaps = 30, kernelSize= [4, 4], activationFun= 'selu')
  cnnModel.addConv2DLayer(featureMaps = 30, kernelSize= [4, 4], activationFun= 'elu')
  cnnModel.addMaxPooling2DLayer(poolingSize= [2,2])
  cnnModel.addDropoutLayer(dropOutRate=0.1)
  cnnModel.addFlattenLayer()
  cnnModel.addMultipleDenseLayers(neuronsAmount=156, layers = 3, activationFun= 'relu')
  opt = keras.optimizers.Adam(learning_rate=0.0015)
  cnnModel.compileModel(lossFun= 'binary_crossentropy', optimizer= opt)
  logging.info(f'Definida taxa de aprendizado de: {round(opt.learning_rate.numpy()*100,2)}%')
  return cnnModel.model

model4 = model4()
model4.summary()

INFO:root:Definida taxa de aprendizado de: 0.15%


Model: "Modelo_04"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_38 (Conv2D)          (None, 30, 24, 24)        780       
                                                                 
 conv2d_39 (Conv2D)          (None, 30, 20, 20)        22530     
                                                                 
 conv2d_40 (Conv2D)          (None, 30, 17, 17)        14430     
                                                                 
 conv2d_41 (Conv2D)          (None, 30, 14, 14)        14430     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 30, 7, 7)         0         
 g2D)                                                            
                                                                 
 dropout_14 (Dropout)        (None, 30, 7, 7)          0         
                                                         

### **Fase 04: Treinamento dos modelos** 


In [ ]:
logging.info('Treinamento do modelo 01')
model1.fit(xTrain, yTrain, validation_data= [xTest, yTest], epochs= 10, batch_size= 200)

INFO:root:Treinamento do modelo 01


Epoch 1/10
300/300 [==============================] - 4s 10ms/step - loss: 0.2965 - accuracy: 0.9021 - val_loss: 0.0899 - val_accuracy: 0.9724
Epoch 2/10
300/300 [==============================] - 3s 10ms/step - loss: 0.0983 - accuracy: 0.9712 - val_loss: 0.0742 - val_accuracy: 0.9792
Epoch 3/10
300/300 [==============================] - 3s 10ms/step - loss: 0.0825 - accuracy: 0.9763 - val_loss: 0.0728 - val_accuracy: 0.9787
Epoch 4/10
300/300 [==============================] - 3s 10ms/step - loss: 0.0699 - accuracy: 0.9803 - val_loss: 0.0596 - val_accuracy: 0.9822
Epoch 5/10
300/300 [==============================] - 3s 10ms/step - loss: 0.0629 - accuracy: 0.9823 - val_loss: 0.0754 - val_accuracy: 0.9811
Epoch 6/10
300/300 [==============================] - 3s 10ms/step - loss: 0.0604 - accuracy: 0.9838 - val_loss: 0.0798 - val_accuracy: 0.9798
Epoch 7/10
300/300 [==============================] - 3s 10ms/step - loss: 0.0599 - accuracy: 0.9842 - val_loss: 0.0665 - val_accuracy: 0.9826

In [ ]:
logging.info('Treinamento do modelo 02')
model2.fit(xTrain, yTrain, validation_data= [xTest, yTest], epochs= 7, batch_size= 75)

INFO:root:Treinamento do modelo 02


Epoch 1/7
800/800 [==============================] - 8s 10ms/step - loss: 1.5047 - accuracy: 0.4936 - val_loss: 0.3470 - val_accuracy: 0.8963
Epoch 2/7
800/800 [==============================] - 8s 9ms/step - loss: 0.3506 - accuracy: 0.8914 - val_loss: 0.1896 - val_accuracy: 0.9429
Epoch 3/7
800/800 [==============================] - 8s 9ms/step - loss: 0.2402 - accuracy: 0.9251 - val_loss: 0.1384 - val_accuracy: 0.9574
Epoch 4/7
800/800 [==============================] - 8s 9ms/step - loss: 0.1974 - accuracy: 0.9385 - val_loss: 0.1162 - val_accuracy: 0.9633
Epoch 5/7
800/800 [==============================] - 8s 9ms/step - loss: 0.1734 - accuracy: 0.9461 - val_loss: 0.1025 - val_accuracy: 0.9696
Epoch 6/7
800/800 [==============================] - 8s 9ms/step - loss: 0.1527 - accuracy: 0.9535 - val_loss: 0.0907 - val_accuracy: 0.9733
Epoch 7/7
800/800 [==============================] - 8s 9ms/step - loss: 0.1411 - accuracy: 0.9563 - val_loss: 0.0826 - val_accuracy: 0.9735


In [ ]:
logging.info('Treinamento do modelo 03')
model3.fit(xTrain, yTrain, validation_data= [xTest, yTest], epochs= 5, batch_size= 150)

INFO:root:Treinamento do modelo 03


Epoch 1/5
400/400 [==============================] - 9s 20ms/step - loss: 0.7383 - accuracy: 0.8755 - val_loss: 0.1209 - val_accuracy: 0.9778
Epoch 2/5
400/400 [==============================] - 7s 18ms/step - loss: 0.0922 - accuracy: 0.9775 - val_loss: 0.0662 - val_accuracy: 0.9812
Epoch 3/5
400/400 [==============================] - 7s 18ms/step - loss: 0.0582 - accuracy: 0.9844 - val_loss: 0.0423 - val_accuracy: 0.9886
Epoch 4/5
400/400 [==============================] - 7s 18ms/step - loss: 0.0440 - accuracy: 0.9878 - val_loss: 0.0379 - val_accuracy: 0.9890
Epoch 5/5
400/400 [==============================] - 7s 18ms/step - loss: 0.0370 - accuracy: 0.9895 - val_loss: 0.0359 - val_accuracy: 0.9885


In [ ]:
logging.info('Treinamento do modelo 04')
model4.fit(xTrain, yTrain, validation_data= [xTest, yTest], epochs= 4, batch_size= 50)

INFO:root:Treinamento do modelo 04


Epoch 1/4
1200/1200 [==============================] - 14s 11ms/step - loss: 0.0331 - accuracy: 0.9479 - val_loss: 0.0128 - val_accuracy: 0.9796
Epoch 2/4
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0129 - accuracy: 0.9800 - val_loss: 0.0135 - val_accuracy: 0.9797
Epoch 3/4
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0108 - accuracy: 0.9835 - val_loss: 0.0097 - val_accuracy: 0.9848
Epoch 4/4
1200/1200 [==============================] - 12s 10ms/step - loss: 0.0089 - accuracy: 0.9868 - val_loss: 0.0092 - val_accuracy: 0.9868


### **Fase 05: Status do modelo treinado** 


In [ ]:
def evaluateModel(model, xTest, yTest, verbose = 0):
  scores = model.evaluate(xTest, yTest, verbose= 0)
  logging.info(f'Accuracy estimated (model {model.name}): {round(scores[1]*100, 2)}%')
  return scores

In [ ]:
evaluateModel(model1, xTest, yTest)
evaluateModel(model2, xTest, yTest)
evaluateModel(model3, xTest, yTest)
evaluateModel(model4, xTest, yTest)

INFO:root:Accuracy estimated (model Modelo_01): 98.19%
INFO:root:Accuracy estimated (model Modelo_02): 97.35%
INFO:root:Accuracy estimated (model Modelo_03): 98.85%
INFO:root:Accuracy estimated (model Modelo_04): 98.68%


[0.009209604933857918, 0.9868000149726868]